In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_ini=pd.read_csv('titanic.csv')

In [3]:
for feature in df_ini.columns[:]:
    if df_ini[feature].dtype!='object':        
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].median(),df_ini[feature])
    else:
        df_ini[feature]=np.where(df_ini[feature].isnull(),df_ini[feature].value_counts().index[0],df_ini[feature])
df_aim=df_ini

In [4]:
def reol(feature1):
    IQR=df_aim[feature1].quantile(0.75)-df_aim[feature1].quantile(0.25)
    lbound=df_aim[feature1].quantile(0.25)-(IQR*1.5)
    ubound=df_aim[feature1].quantile(0.75)+(IQR*1.5)
    df_aim.loc[df_aim[feature1]<=lbound,feature1]=lbound
    df_aim.loc[df_aim[feature1]>=ubound,feature1]=ubound
df_aol=df_aim

In [5]:
for feature in df_aim.columns[:]:
    if df_aim[feature].dtype!='object':
        reol(feature)

In [6]:
df_aol_cont1=df_aol.select_dtypes(exclude=[object])

In [7]:
df_aol_cont2=df_aol_cont1.drop(columns='Pclass',axis=1) #Dropping categorical ordinal features

In [8]:
df_aol_cont3=df_aol_cont2.drop('Survived',axis=1)#varience threshold can be applied for independent features only

In [9]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)
var_thres.fit(df_aol_cont3)

VarianceThreshold(threshold=0)

In [10]:
const_features=[column for column in df_aol_cont3.columns\
               if column not in df_aol_cont3.columns[var_thres.get_support()]]

In [11]:
df_aol=df_aol.drop(const_features,axis=1)#Dropping constant features

In [12]:
df_aol_cont3=df_aol_cont3.drop(columns=const_features,axis=1)

In [13]:
X=df_aol_cont3
y=df_aol_cont2['Survived']

In [14]:
def pcc(dataset,threshold):
    dropset=set()
    cor_mat=dataset.corr()
    for i in range(len(cor_mat.columns)):
        for j in range(i):
            if abs(cor_mat.iloc[i,j])>=threshold:
                cor_feat=cor_mat.columns[i]
                dropset.add(cor_feat)
    return dropset 
dropset1=pcc(X,0.85)
print("The Features to be Dropped are...")
dropset1

The Features to be Dropped are...


set()

In [15]:
df_aol=df_aol.drop(columns=dropset1,axis=1)

In [16]:
def enc(feature1):
    key_labels=df_aol[feature1].unique()
    dict={k:i for i,k in enumerate(key_labels,0)}
    df_aol[feature1]=df_aol[feature1].map(dict)

In [17]:
for feature in df_aol.columns[:]:
    if df_aol[feature].dtype=='object':
        enc(feature)
df_aec=df_aol

In [18]:
from sklearn.feature_selection import mutual_info_classif
X=df_aec.drop(columns='Survived',axis=1)
y=df_aec['Survived']
mut_inf_data=mutual_info_classif(X,y,random_state=0)
mut_inf_modif=pd.Series(mut_inf_data,index=X.columns)
mut_inf_sorted=mut_inf_modif.sort_values(ascending=False)
dict=mut_inf_sorted.to_dict()

In [19]:
droplist=[]
for key in dict:
    if dict[key]<0.001:
        droplist.append(key)

In [20]:
df_aec=df_aec.drop(columns=droplist,axis=1)

In [21]:
df_aim_cate1=df_aim.select_dtypes(include=[object])#For imbalanced data check,include all categorical features+Dep. feature

In [22]:
df_aim_cate2=df_aim[['Pclass','Survived']].copy()#include categorical ordinal feature and
df_aim_cate=pd.concat([df_aim_cate1,df_aim_cate2],axis=1)#dependent feature in the new Data frame.

In [23]:
imb_chk_list=list()

In [24]:
for feature in df_aim_cate.columns[:]:
    length=int(len(df_aim_cate[feature].unique()))      
    if length <= 5:
        imb_chk_list.append(feature)

In [25]:
drop_list=[]#Add the independent features with single category, imbalanced data in this list(Difference of values<0.05 and >0.95)

In [26]:
df_aec=df_aec.drop(drop_list,axis=1)

In [27]:
original_list=['Survived','Pclass','Sex','SibSp','Cabin','Embarked']
log_list=['Fare']
expo_list=[]
sqroot_list=['PassengerId','Name','Ticket']
boxcox_list=[]

In [28]:
for feature in log_list:
    df_aec[feature]=np.log1p(df_aec[feature])

In [29]:
for feature in sqroot_list:
    df_aec[feature]=df_aec[feature]**(1/2)

In [30]:
df_atr=df_aec

In [31]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaled=scaler.fit_transform(df_atr)

In [32]:
df_ast=pd.DataFrame(scaled,columns=df_atr.columns)

In [33]:
df_fin=df_ast

In [34]:
X=df_fin.drop(columns='Survived',axis=1)
y=df_fin['Survived']

In [35]:
from sklearn.model_selection import train_test_split

In [36]:
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.7,random_state=0)

In [37]:
import xgboost
classifier=xgboost.XGBClassifier()
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:33:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [38]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[149  19]
 [ 30  70]]
Accuracy Score : 0.8171641791044776
              precision    recall  f1-score   support

         0.0       0.83      0.89      0.86       168
         1.0       0.79      0.70      0.74       100

    accuracy                           0.82       268
   macro avg       0.81      0.79      0.80       268
weighted avg       0.82      0.82      0.81       268



In [39]:
from sklearn.model_selection import GridSearchCV
classifier=xgboost.XGBClassifier()

In [40]:
param_grid={'learning_rate':[0.05,0.10,0.15,0.20,0.25,0.30],
            'max_depth':[3,4,5,6,8,10,12],
            'min_child_weight':[1,3,5,7],
            'gamma':[0.0,0.1,0.2,0.3,0.4],
            'colsample_bytree':[0.3,0.4,0.5,0.7]}            

In [42]:
grid_search=GridSearchCV(estimator=classifier,param_grid=param_grid,cv=10,
                         verbose=2,n_jobs=-1)

In [43]:
grid_search.fit(X_train,y_train)

Fitting 10 folds for each of 3360 candidates, totalling 33600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:   35.4s
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:   53.5s
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 4893 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 5824 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 6837 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 7930 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 9105 tasks      | 

[19:17:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_job...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, va

In [44]:
best_grid=grid_search.best_estimator_
best_grid

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=4,
              min_child_weight=7, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [45]:
prediction=best_grid.predict(X_test)

In [46]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[148  20]
 [ 31  69]]
Accuracy Score : 0.8097014925373134
              precision    recall  f1-score   support

         0.0       0.83      0.88      0.85       168
         1.0       0.78      0.69      0.73       100

    accuracy                           0.81       268
   macro avg       0.80      0.79      0.79       268
weighted avg       0.81      0.81      0.81       268



In [64]:
# pip install optuna
import optuna
from sklearn.model_selection import cross_val_score
def objective(trial):
    learning_rate=trial.suggest_categorical('learning_rate',[0.05,0.10,0.15,0.20,0.25,0.30])
    max_depth=trial.suggest_categorical('max_depth',[3,4,5,6,8,10,12])
    min_child_weight=trial.suggest_categorical('min_child_weight',[1,3,5,7])
    gamma=trial.suggest_categorical('gamma',[0.0,0.1,0.2,0.3,0.4])
    colsample_bytree=trial.suggest_categorical('colsample_bytree',[0.3,0.4,0.5,0.7])
    classifier=xgboost.XGBClassifier(learning_rate=learning_rate,max_depth=max_depth,min_child_weight=min_child_weight,
               gamma=gamma,colsample_bytree=colsample_bytree)
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=3).mean()
    return score

In [65]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)

[I 2021-07-29 21:47:43,504] A new study created in memory with name: no-name-d5413337-169e-43fd-8309-26e204686508
[I 2021-07-29 21:47:46,729] Trial 0 finished with value: 0.7753081258516041 and parameters: {'learning_rate': 0.3, 'max_depth': 12, 'min_child_weight': 7, 'gamma': 0.0, 'colsample_bytree': 0.7}. Best is trial 0 with value: 0.7753081258516041.
[I 2021-07-29 21:47:48,868] Trial 1 finished with value: 0.7704926916883439 and parameters: {'learning_rate': 0.05, 'max_depth': 10, 'min_child_weight': 3, 'gamma': 0.3, 'colsample_bytree': 0.4}. Best is trial 0 with value: 0.7753081258516041.
[I 2021-07-29 21:47:49,030] Trial 2 finished with value: 0.7833673974978322 and parameters: {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'gamma': 0.0, 'colsample_bytree': 0.4}. Best is trial 2 with value: 0.7833673974978322.
[I 2021-07-29 21:47:49,202] Trial 3 finished with value: 0.7576566951566952 and parameters: {'learning_rate': 0.3, 'max_depth': 4, 'min_child_weight': 1, 'g

[I 2021-07-29 21:47:54,572] Trial 32 finished with value: 0.794577604360213 and parameters: {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.1, 'colsample_bytree': 0.7}. Best is trial 12 with value: 0.794577604360213.
[I 2021-07-29 21:47:54,735] Trial 33 finished with value: 0.7849312523225566 and parameters: {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 3, 'gamma': 0.3, 'colsample_bytree': 0.7}. Best is trial 12 with value: 0.794577604360213.
[I 2021-07-29 21:47:54,935] Trial 34 finished with value: 0.773713303604608 and parameters: {'learning_rate': 0.3, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.1, 'colsample_bytree': 0.7}. Best is trial 12 with value: 0.794577604360213.
[I 2021-07-29 21:47:55,102] Trial 35 finished with value: 0.7817338659729964 and parameters: {'learning_rate': 0.05, 'max_depth': 4, 'min_child_weight': 5, 'gamma': 0.0, 'colsample_bytree': 0.4}. Best is trial 12 with value: 0.794577604360213.
[I 2021-07-29 21:47:55,284]

[I 2021-07-29 21:48:00,570] Trial 64 finished with value: 0.794577604360213 and parameters: {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.1, 'colsample_bytree': 0.7}. Best is trial 12 with value: 0.794577604360213.
[I 2021-07-29 21:48:00,790] Trial 65 finished with value: 0.7833209463644245 and parameters: {'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 5, 'gamma': 0.1, 'colsample_bytree': 0.7}. Best is trial 12 with value: 0.794577604360213.
[I 2021-07-29 21:48:00,959] Trial 66 finished with value: 0.7849389941781246 and parameters: {'learning_rate': 0.2, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.3, 'colsample_bytree': 0.7}. Best is trial 12 with value: 0.794577604360213.
[I 2021-07-29 21:48:01,161] Trial 67 finished with value: 0.7656695156695156 and parameters: {'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1, 'gamma': 0.1, 'colsample_bytree': 0.4}. Best is trial 12 with value: 0.794577604360213.
[I 2021-07-29 21:48:01,368

[I 2021-07-29 21:48:06,616] Trial 96 finished with value: 0.7704694661216399 and parameters: {'learning_rate': 0.15, 'max_depth': 6, 'min_child_weight': 5, 'gamma': 0.0, 'colsample_bytree': 0.7}. Best is trial 77 with value: 0.7961801684627772.
[I 2021-07-29 21:48:06,792] Trial 97 finished with value: 0.7961801684627772 and parameters: {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.0, 'colsample_bytree': 0.7}. Best is trial 77 with value: 0.7961801684627772.
[I 2021-07-29 21:48:06,970] Trial 98 finished with value: 0.775315867707172 and parameters: {'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 5, 'gamma': 0.0, 'colsample_bytree': 0.3}. Best is trial 77 with value: 0.7961801684627772.
[I 2021-07-29 21:48:07,133] Trial 99 finished with value: 0.7961801684627772 and parameters: {'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 5, 'gamma': 0.0, 'colsample_bytree': 0.7}. Best is trial 77 with value: 0.7961801684627772.


In [66]:
best=study.best_params

In [67]:
best

{'learning_rate': 0.05,
 'max_depth': 3,
 'min_child_weight': 5,
 'gamma': 0.0,
 'colsample_bytree': 0.7}

In [68]:
classifier=xgboost.XGBClassifier(learning_rate=best['learning_rate'],max_depth=best['max_depth'],
           min_child_weight=best['min_child_weight'],gamma=best['gamma'],colsample_bytree=best['colsample_bytree'])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

[21:48:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [69]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[148  20]
 [ 28  72]]
Accuracy Score : 0.8208955223880597
              precision    recall  f1-score   support

         0.0       0.84      0.88      0.86       168
         1.0       0.78      0.72      0.75       100

    accuracy                           0.82       268
   macro avg       0.81      0.80      0.81       268
weighted avg       0.82      0.82      0.82       268



In [71]:
# pip install hyperopt
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
space={'learning_rate':hp.choice('learning_rate',[0.05,0.10,0.15,0.20,0.25,0.30]),
       'max_depth':hp.choice('max_depth',[3,4,5,6,8,10,12]),
       'min_child_weight':hp.choice('min_child_weight',[1,3,5,7]),
       'gamma':hp.choice('gamma',[0.0,0.1,0.2,0.3,0.4]),
       'colsample_bytree':hp.choice('colsample_bytree',[0.3,0.4,0.5,0.7])}

In [72]:
def objective(space):
    classifier=xgboost.XGBClassifier(learning_rate=space['learning_rate'],max_depth=space['max_depth'],
               min_child_weight=space['min_child_weight'],gamma=space['gamma'],colsample_bytree=space['colsample_bytree'])
    score=cross_val_score(classifier,X_train,y_train,n_jobs=-1,cv=5).mean()
    return{'loss':-score,'status':STATUS_OK}

In [73]:
trials=Trials()
best=fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=80,trials=trials)
best

100%|███████████████████████████████████████████████| 80/80 [00:35<00:00,  2.23trial/s, best loss: -0.7945935483870968]


{'colsample_bytree': 0,
 'gamma': 1,
 'learning_rate': 2,
 'max_depth': 5,
 'min_child_weight': 3}

In [86]:
lerat={0:0.05,1:0.10,2:0.15,3:0.20,4:0.25,5:0.30}
maxdep={0:3,1:4,2:5,3:6,4:8,5:10,6:12}
minchwe={0:1,1:3,2:5,3:7}
gam={0:0.0,1:0.1,2:0.2,3:0.3,4:0.4}
colsambyt={0:0.3,1:0.4,2:0.5,3:0.7}

In [88]:
classifier=xgboost.XGBClassifier(learning_rate=lerat[best['learning_rate']],
            max_depth=maxdep[best['max_depth']],min_child_weight=minchwe[best['min_child_weight']],
            gamma=gam[best['gamma']],colsample_bytree=colsambyt[best['colsample_bytree']])
classifier.fit(X_train,y_train)
prediction=classifier.predict(X_test)

[22:30:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [90]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print("Accuracy Score :",accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[143  25]
 [ 30  70]]
Accuracy Score : 0.7947761194029851
              precision    recall  f1-score   support

         0.0       0.83      0.85      0.84       168
         1.0       0.74      0.70      0.72       100

    accuracy                           0.79       268
   macro avg       0.78      0.78      0.78       268
weighted avg       0.79      0.79      0.79       268

